# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from utils.loading import load_table

import pandas as pd
import plotly.express as px
import numpy as np

import datetime
pd.set_option('display.max_rows', 1000)

# Data Loading
- Load insuline related tables 
- Create subsets for a certain passenger and (randomized) day

In [2]:
basal_bolus      = load_table('MonitorBasalBolus',     date_cols=['LocalDeliveredDtTm'])
correction_bolus = load_table('MonitorCorrectionBolus',date_cols=['LocalDeliveredDtTm'])
meal             = load_table('MonitorMeal',           date_cols=['LocalDtTm'])
meal_bolus       = load_table('MonitorMealBolus',      date_cols=['LocalDeliveredDtTm'])
total_bolus      = load_table('MonitorTotalBolus',     date_cols=['LocalDeliveredDtTm'])
pump             = load_table('Pump',                  date_cols=['DataDtTm'])
system           = load_table('MonitorSystem',         date_cols=['LocalDtTm'])



In [3]:
DeidentID = 20
date = np.random.choice(basal_bolus [basal_bolus.DeidentID == DeidentID].LocalDeliveredDtTm.dt.date.unique())

# Manual override
date = datetime.date(2014,2,20)
print(date)


2014-02-20


In [4]:

basal_bolus_x = basal_bolus [(basal_bolus.DeidentID == DeidentID)&(basal_bolus.LocalDeliveredDtTm.dt.date == date)]
correction_bolus_x = correction_bolus [(correction_bolus.DeidentID == DeidentID)&(correction_bolus.LocalDeliveredDtTm.dt.date == date)]
meal_bolus_x = meal_bolus [(meal_bolus.DeidentID == DeidentID)&(meal_bolus.LocalDeliveredDtTm.dt.date == date)]
total_bolus_x = total_bolus [(total_bolus.DeidentID == DeidentID)&(total_bolus.LocalDeliveredDtTm.dt.date == date)]
meal_x = meal [(meal.DeidentID == DeidentID)&(meal.LocalDtTm.dt.date == date)]
pump_x = pump [(pump.DeidentID == DeidentID)&(pump.DataDtTm.dt.date == date)]
system_x = system[(system.DeidentID == DeidentID)&(system.LocalDtTm.dt.date == date)]


In [5]:
print(sum(correction_bolus_x.DeliveredValue))
print(sum(meal_bolus_x.DeliveredValue))
print(sum(basal_bolus_x.DeliveredValue))
print(sum(total_bolus_x.DeliveredValue))

print(sum(correction_bolus_x.DeliveredValue )+sum(meal_bolus_x.DeliveredValue )+sum(basal_bolus_x.DeliveredValue ))



17.700000047683716
17.199999690055847
24.70000039041042
59.60000013560057
59.600000128149986


# Look at one passenger-day in detail

## Notes


2014-02-03 / 20: 
- Time of meal bolus corresponds totime of manual correction boluses for 3/4 meals. 
- In those cases, the “total” is the sum of the two values. 
- Totals add up: Total = meal+correction+basal (this is different to documentation!)

Almost all of the times, this adds up super well. 

2014-04-17 / 20: 
- There is one injection in the full-outer-join that does not add up: 2.3 have been given at 15:43:28 but that that is not accounted for in any other tables. 

### Systematic check: 
- We can reproduce the value in about 95% of patient-days. 
- most of the times when we see differences, the value in “total_bolus”-table seems to small compared to manual calculation.


2014-09-03 / 2: 
- Checks out throughout the day there’s only one occasion (06:28:05), where correction=0.9, delivered=3.3 and total only 0.2, which exactly explains the differences of 4.0. mode in that time _should_ be normal operation. 

Look for values with mismatches and with pump data
2013-09-03 / 20: 
- Pump shows the 0.2 that can be found in _correction, but not in _total. 


2014-07-16 / 3: 
- User switched between system states 0 and 1. 
- First glance: when system status is 0, the entries in “correction_bolus” do not show up in total bolus. Switched between status 0 and 1 quite a lot. 



### mismatches pump vs. total: 
2013-11-23 / 2: 
- Pump was deactivated during the day. 


2013-03-10 / 28: 
- Only one correction entry (7.0 at 14:42:37) does not show up in the totals table and it also does not show up in the pump table. the status at this point in time was 0. 
- 

2012-06-19 / 4: 
- Pump data ends early. 

### Other observations: 
- If the diff is positive (meaning the total is higher tahn the components), it’s typically a very small amount. Let’s check one counterexample: 16 / 2012-08-11
  - reason: there are duplicated values in the total column. Makes me like that table even less. 


2013-11-14 / 2
- pump and monitor can be a few (I’ve only seen 1) seconds apart. super annoying to fix in pandas. 




In [6]:
# System table
system_x = system_x.sort_values('LocalDtTm')
# Only see changes (condenses table quite a bit)
system_x[system_x.DiAsState !=system_x['DiAsState'].shift(1)]

# Uncomment to see full table. 
#system_x

,DeidentID,RecID,LocalDtTm,SafetyMode,DiAsState,Battery,IOBValue,Hypolight,Hyperlight,Exercising,AlarmNoCGM,AlarmHypo
1803235,20,3914162,2014-02-20 00:02:11,True,2,86,0.026486,0,0,False,False,False
1802782,20,3913709,2014-02-20 11:19:37,True,3,84,3.026950,0,0,False,False,False
1802740,20,3913667,2014-02-20 12:21:09,True,2,76,0.900580,1,0,False,False,False


In [7]:
# Correction bolus
correction_bolus_x.sort_values('LocalDeliveredDtTm')

,DeidentID,RecID,DeliveredValue,BolusSource,LocalDeliveredDtTm
16237,20,30600,1.0,Manual,2014-02-20 10:44:12
16236,20,30599,1.0,Manual,2014-02-20 11:16:06
16235,20,30598,0.3,Manual,2014-02-20 12:56:33
16234,20,30597,1.0,Manual,2014-02-20 14:13:49
16233,20,30596,1.0,Manual,2014-02-20 14:36:25
16232,20,30595,1.5,Manual,2014-02-20 15:24:38
16231,20,30594,0.6,Automated,2014-02-20 16:27:39
16230,20,30593,0.8,Automated,2014-02-20 17:32:40
16229,20,30592,1.0,Manual,2014-02-20 19:22:12
16228,20,30591,2.0,Manual,2014-02-20 19:26:56


In [8]:
# Meal Bolus
meal_bolus_x.sort_values('LocalDeliveredDtTm')

,DeidentID,RecID,DeliveredValue,LocalDeliveredDtTm
10138,20,19196,1.6,2014-02-20 10:44:12
10137,20,19195,5.0,2014-02-20 12:56:33
10136,20,19194,3.1,2014-02-20 13:55:24
10135,20,19193,5.7,2014-02-20 19:51:38
10134,20,19192,1.8,2014-02-20 20:55:06


In [9]:
# Total bolus (commented out as the output is very long)
#total_bolus_x.sort_values('LocalDeliveredDtTm')

In [10]:
# Basal bolus bolus (commented out as the output is very long)
# basal_bolus_x.sort_values('LocalDeliveredDtTm')

In [11]:
# Most of the total- and basal- tables are identical. 
# Filter on occurrences where they _differ_. 
temp = basal_bolus_x.merge(total_bolus_x, on='LocalDeliveredDtTm', how='outer', suffixes=('_basal', '_total'))
temp [temp.DeliveredValue_basal != temp.DeliveredValue_total].sort_values('LocalDeliveredDtTm')

,DeidentID_basal,RecID_basal,DeliveredValue_basal,LocalDeliveredDtTm,DeidentID_total,RecID_total,DeliveredValue_total
220,NaN,NaN,NaN,2014-02-20 10:44:12,20,882839,2.6
219,NaN,NaN,NaN,2014-02-20 11:16:06,20,882833,1.0
81,20.0,855482.0,6.661340e-16,2014-02-20 12:56:33,20,882823,5.3
218,NaN,NaN,NaN,2014-02-20 13:55:24,20,882819,3.1
217,NaN,NaN,NaN,2014-02-20 14:13:49,20,882815,1.0
216,NaN,NaN,NaN,2014-02-20 14:36:25,20,882810,1.0
215,NaN,NaN,NaN,2014-02-20 15:24:38,20,882801,1.5
56,20.0,855457.0,1.000000e-01,2014-02-20 16:27:39,20,882794,0.7
48,20.0,855449.0,1.000000e-01,2014-02-20 17:32:40,20,882786,0.9
214,NaN,NaN,NaN,2014-02-20 19:22:12,20,882776,1.0


In [12]:
# Pump values
# this table is often empty -- see data coverage plot for a better overview. 
pump_x.sort_values('DataDtTm')

,DeidentID,RecID,DataDtTm,Pump,BasalRt,InsulinTDD,DataDtTmAdjusted


In [13]:
# SYSTEMATIC CHECK. 

# Aggregate basal on date/patient level
basal_bolus['date'] = basal_bolus['LocalDeliveredDtTm'].dt.date
basal_sums = basal_bolus.groupby(['DeidentID', 'date'], as_index=False).agg(DeliveredValue=('DeliveredValue', sum), start_time_basal=('LocalDeliveredDtTm',min), end_time_basal=('LocalDeliveredDtTm',max))

# Aggregate meal on date/patient level
meal_bolus['date'] = meal_bolus['LocalDeliveredDtTm'].dt.date
meal_sums = meal_bolus.groupby(['DeidentID', 'date'], as_index=False).DeliveredValue.sum()

# Aggregate correction on date/patient level
correction_bolus['date'] = correction_bolus['LocalDeliveredDtTm'].dt.date
correction_sums = correction_bolus.groupby(['DeidentID', 'date'], as_index=False).DeliveredValue.sum()

# Aggregate total on date/patient level
total_bolus['date'] = total_bolus['LocalDeliveredDtTm'].dt.date
total_sums = total_bolus.groupby(['DeidentID', 'date'], as_index=False).DeliveredValue.sum()


# Aggregate pump data on date/patient level and get some stats to indicate whether we have a full day of data.
pump['date'] = pump['DataDtTm'].dt.date
pump['time'] = pump['DataDtTm'].dt.time
pump_sums = pump.groupby(['DeidentID', 'date'], as_index=False).agg(pump_sum=('Pump', sum),insulin_total= ('InsulinTDD', sum), start_time_pump=('DataDtTm',min), end_time_pump=('DataDtTm',max))


In [14]:
# Put it all together
sums = basal_sums.merge(meal_sums, on=['DeidentID', 'date'], how='left', suffixes=('_basal', '_meal')).merge(correction_sums, on=['DeidentID', 'date'], how='left').merge(total_sums, on=['DeidentID', 'date'], how='left', suffixes=('_correction', '_total'))
sums['total_reproduced'] = sums.DeliveredValue_meal.fillna(0) + sums.DeliveredValue_basal.fillna(0) + sums.DeliveredValue_correction.fillna(0)
sums = sums.merge(pump_sums, on=['DeidentID', 'date'], how='left')
sums['total_identical'] = abs(sums.DeliveredValue_total - sums.total_reproduced)<1e-5

print(f'Matchrate of "total" vs. "sum of 3 components": {sums.total_identical.mean()}')

# Get the pump data in. 
sums.groupby('DeidentID').total_identical.mean()
sums['full_day_pump'] = (sums.end_time_pump - sums.start_time_pump) > datetime.timedelta(hours=23)
sums['full_day_basal'] = (sums.end_time_basal - sums.start_time_basal) > datetime.timedelta(hours=23)

sums = sums.assign(diff_total_values = sums.DeliveredValue_total - sums.total_reproduced, pump_diff = sums.pump_sum - sums.insulin_total)


Matchrate of "total" vs. "sum of 3 components": 0.9534424379232506


In [15]:
print('Total and 3 components do not match: ')
sums[~sums.total_identical].head(10)


Total and 3 components do not match: 


,DeidentID,date,DeliveredValue_basal,start_time_basal,end_time_basal,DeliveredValue_meal,DeliveredValue_correction,DeliveredValue_total,total_reproduced,pump_sum,insulin_total,start_time_pump,end_time_pump,total_identical,full_day_pump,full_day_basal,diff_total_values,pump_diff
51,2,2013-11-09,3.2,2013-11-09 12:24:20,2013-11-09 23:29:20,12.0,14.600000,19.9,29.800000,NaN,NaN,NaT,NaT,False,False,False,-9.900000,NaN
52,2,2013-11-10,6.3,2013-11-10 02:09:21,2013-11-10 23:04:21,31.0,9.900000,28.3,47.200000,NaN,NaN,NaT,NaT,False,False,False,-18.900000,NaN
53,2,2013-11-11,5.9,2013-11-11 00:19:21,2013-11-11 21:10:02,32.5,13.800000,35.2,52.200000,NaN,NaN,NaT,NaT,False,False,False,-17.000000,NaN
54,2,2013-11-12,4.3,2013-11-12 03:24:18,2013-11-12 20:18:04,32.5,10.400000,19.2,47.200000,NaN,NaN,NaT,NaT,False,False,False,-28.000000,NaN
55,2,2013-11-13,4.9,2013-11-13 03:04:39,2013-11-13 21:08:54,30.9,9.400000,17.5,45.200000,NaN,NaN,NaT,NaT,False,False,False,-27.699999,NaN
56,2,2013-11-14,2.9,2013-11-14 03:58:53,2013-11-14 18:48:54,35.4,11.100000,28.3,49.400000,65.7,65.9,2013-11-14,2013-11-14 23:59:59,False,True,False,-21.100000,-0.2
57,2,2013-11-15,4.8,2013-11-15 00:08:54,2013-11-15 20:08:54,32.7,12.100000,20.9,49.600000,65.1,65.2,2013-11-15,2013-11-15 23:59:59,False,True,False,-28.700000,-0.1
58,2,2013-11-16,2.3,2013-11-16 03:58:54,2013-11-16 12:46:30,3.5,37.500000,6.2,43.300000,59.4,69.1,2013-11-16,2013-11-16 23:59:59,False,True,False,-37.100000,-9.7
59,2,2013-11-21,1.5,2013-11-21 16:12:31,2013-11-21 23:58:53,8.0,2.500000,10.4,12.000000,45.7,57.7,2013-11-21,2013-11-21 23:59:59,False,True,False,-1.600000,-12.0
60,2,2013-11-22,5.9,2013-11-22 00:03:54,2013-11-22 23:41:52,17.1,12.733333,11.8,35.733333,61.1,61.1,2013-11-22,2013-11-22 23:59:59,False,True,True,-23.933333,0.0


## Phase 2: Focusing more on the pump table. 

In [16]:
basal_bolus_x.sort_values('LocalDeliveredDtTm').head(2)

,DeidentID,RecID,DeliveredValue,LocalDeliveredDtTm
453162,20,855609,0.2,2014-02-20 00:02:36
453161,20,855608,0.1,2014-02-20 00:07:36



df = pump_x[[ 'DeidentID', 'DataDtTm', 'Pump', 'BasalRt', 'InsulinTDD']]\
.merge(meal_bolus_x[[ 'DeidentID', 'LocalDeliveredDtTm', 'DeliveredValue']], left_on=[ 'DeidentID', 'DataDtTm'], right_on=[ 'DeidentID', 'LocalDeliveredDtTm'] , how='outer')\
.merge(basal_bolus_x[[ 'DeidentID', 'LocalDeliveredDtTm', 'DeliveredValue']], left_on=[ 'DeidentID', 'DataDtTm'], right_on=[ 'DeidentID', 'LocalDeliveredDtTm'] , how='outer')\
.merge(correction_bolus_x[[ 'DeidentID', 'LocalDeliveredDtTm', 'DeliveredValue']], left_on=[ 'DeidentID', 'DataDtTm'], right_on=[ 'DeidentID', 'LocalDeliveredDtTm'] , how='outer')\


#df.sort_values('DataDtTm')

In [17]:
# Split pump data into three components 
# (basically a database normalization)
pump_totals = pump[~pump.InsulinTDD.isna()]
pump_rates = pump[~pump.BasalRt.isna()]
pump_shots = pump[~pump.Pump.isna()]
pump_rates = pump_rates.sort_values(['DeidentID', 'DataDtTm'])

# Shift pump rates by one which will allow us to calculate time spans. 
pump_rates[['shifted_data_dt_tm', 'shifted_DeidentID']] = pump_rates.shift(-1)[['DataDtTm', 'DeidentID']]
pump_rates['Timespan'] = pump_rates.shifted_data_dt_tm-pump_rates.DataDtTm 
# Nullify data where we calculate the time between different patients. 
pump_rates.where(pump_rates.shifted_DeidentID != pump_rates.DeidentID, None)

# Calculate timespans. Only keep rows with hourly data to get rid of weird mismatches. 
# Also, we want to only look at data where we have the full day covered for now. 
pump_rates['Minutes'] = pump_rates.Timespan.dt.total_seconds()/60
pump_rates = pump_rates [pump_rates['Minutes'] <= 61]
sum_minutes = pump_rates.groupby(['DeidentID', 'date'], as_index=False).Minutes.sum()
full_days = sum_minutes[abs(sum_minutes.Minutes-1440)<1e-8]

pump_rates = pump_rates.merge(full_days, on=['DeidentID', 'date'])

# Calculate the actual units for the rates-table and sum up everything. 
pump_rates['Basal_units'] = pump_rates.Minutes_x/60 * pump_rates.BasalRt
pump_rates_final = pump_rates.groupby(['DeidentID', 'date'], as_index=False).Basal_units.sum()
pump_shots_final = pump_shots.groupby(['DeidentID', 'date'], as_index=False).Pump.sum()
pump_totals_final = pump_totals.groupby(['DeidentID', 'date'], as_index=False).InsulinTDD.sum()


In [18]:
# Put the three tables back together and check mismatches. 
merged = pump_rates_final.merge(pump_shots_final, on=['DeidentID', 'date']).merge(pump_totals_final, on=['DeidentID', 'date'])
merged['check'] = merged['Basal_units'] + merged['Pump']
merged['match_yn'] = abs(merged['check'] - merged['InsulinTDD'])<0.1
merged.groupby('DeidentID').match_yn.mean()
# There's a clear difference between patients!

DeidentID
1     0.227273
2     0.444444
3     0.148148
4     0.545455
5     0.406250
6     0.588235
7     0.268293
8     0.424242
9     0.666667
10    0.208955
11    0.750000
12    0.793103
13    0.564103
14    0.425000
15    0.481481
16    0.547619
17    0.491803
18    0.775000
19    0.782609
20    0.705882
21    0.380952
22    0.590909
23    0.555556
24    0.619048
25    0.000000
26    0.727273
27    0.787234
28    0.627907
29    0.368421
30    0.648649
Name: match_yn, dtype: float64

In [19]:
# Check specific mismatches. 
# Remove .head(10) if you need to look at the entire table!
system[(system.DeidentID==2)&(system.LocalDtTm.dt.date == datetime.date(2014,8,13))].sort_values('LocalDtTm').head(10)

,DeidentID,RecID,LocalDtTm,SafetyMode,DiAsState,Battery,IOBValue,Hypolight,Hyperlight,Exercising,AlarmNoCGM,AlarmHypo
1581474,2,3692401,2014-08-13 00:03:32,True,2,47,1.179700,0,0,False,False,False
1581473,2,3692400,2014-08-13 00:03:36,True,2,47,1.179700,1,0,False,False,False
1581472,2,3692399,2014-08-13 00:04:14,True,2,47,1.179700,1,0,False,False,False
1581471,2,3692398,2014-08-13 00:08:32,True,2,52,1.088720,1,0,False,False,False
1581470,2,3692397,2014-08-13 00:09:14,True,2,52,1.088720,1,0,False,False,False
1581469,2,3692396,2014-08-13 00:13:32,True,2,57,1.005140,1,0,False,False,False
1581468,2,3692395,2014-08-13 00:13:36,True,2,57,1.005140,0,0,False,False,False
1581467,2,3692394,2014-08-13 00:14:14,True,2,57,1.005140,0,0,False,False,False
1581466,2,3692393,2014-08-13 00:18:33,True,2,62,0.829444,0,0,False,False,False
1581465,2,3692392,2014-08-13 00:19:14,True,2,63,0.829444,0,0,False,False,False


# Next steps: 
- Maybe we can find a pattern that explains the mismatches we see based on the status table?
- Potentially build one comprehensive table based on the three components (all in all, this seems to be the best source as of now)
- ...